In [1]:
import gym
import tensorflow as tf
import tensorflow.contrib.slim as slim
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from tensorflow import keras
from ns3gym import ns3env

/home/sheila/anaconda3/envs/environment_py3_7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sheila/anaconda3/envs/environment_py3_7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/sheila/anaconda3/envs/environment_py3_7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/sheila/

In [2]:
# ---- m_collisionTh = 11 

env = gym.make('ns3-v0') # environment ID 

# numero de estados
s_num = env.observation_space.shape[0]

# numero de acoes
a_num = env.action_space.n

print("\n")
print("Numero de Estados: ", s_num)
print("Numero de Acoes: ", a_num) 


model = keras.Sequential()
model.add(keras.layers.Dense(s_num, input_shape=(s_num,), activation='relu'))
model.add(keras.layers.Dense(a_num, activation='softmax'))
model.compile(optimizer=tf.train.AdamOptimizer(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

Got new port for ns3gm interface:  8675


Numero de Estados:  4
Numero de Acoes:  4
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [3]:
def choose_action(s, s_num, epsilon):
# Selecione a ação com o maior valor Q.
    if np.random.uniform() < epsilon:
        a = int(np.argmax(model.predict(s)[0][0]))
# Selecione uma ação aleatóriamente.
    else:
        a = np.random.randint(0, s_num)
    return a

In [4]:
numTimeSlots = 98
env._max_episode_steps = numTimeSlots

discount_rate = 0.95  # gamma
# learning rate
learning_rate0 = 0.05 # 0.05 # alpha
learning_rate_decay = 0.1 # 0.1 alpha_decay

# exploration rate
epsilon = 0.5               
epsilon_min = 0.01
epsilon_decay = 0.999

# s - Current State
s = env.reset()
s = np.reshape(s, [1, s_num])


for iteration in range(0,numTimeSlots):
    
    
    # a - Action   
    a = choose_action(s, s_num, epsilon) # acao
    #print(isinstance(a, int))
    
    # Step
    next_state, reward, done, _ = env.step(a) # toma uma ação  no ambiente a cada passo de tempo
    #print("current state: ", s, "next state: ", next_state)
    
    # o Next State retornado do env fica no formato [0, 0, 0, 0]
    # é necesário que o Next State fique no formato de vector sem virgula. 
    #next_state = np.array(next_state)
    #next_state = np.reshape(next_state, [1, s_num])
    #print("next state array: ", next_state)
        
    s_index_num = np.array(s)
    n_s_index_num = np.array(next_state)
    
    # state index
    s_index   = np.argwhere(s_index_num==1)[0][0] 
        
    # next-state index
    n_s_index   = np.argwhere(n_s_index_num==1)[0][0]
    
    next_state = np.reshape(next_state, [1, s_num])        
        
    # next-state Q-valor Maximo
    #n_s_Qmax = np.max(Q_table[np.argwhere(next_state==1)[0][0]]) 
    #n_s_Qmax = np.max(Q_table[n_s_index])
    
       
    print("Time Slot    :", iteration)
    print("----------------------", "\n")
    print("Current State:", s)
    print("Action       :", a)
    print("Next state   :", next_state) #, "-->", (s + 1) % 4
    print("Reward       :", reward)
    print("Gameover     :", done)
    print("----------------------", "\n")   
       
    """   
    learning_rate = learning_rate0 / (1 + iteration * learning_rate_decay)# passo de aprendizagem
    
    # Calculates and updates the Q-table values
    Q_table[s_index, a] = (1 - learning_rate) * Q_table[s_index, a] + learning_rate * (
        reward + discount_rate * n_s_Qmax)
    
    # updates the current state
    s = next_state
    """
    
    target = (reward + discount_rate * np.amax(model.predict(next_state)[0][0]))
    target_f = model.predict(s)
    target_f[0][a] = target
    model.fit(s, target_f, epochs=1, verbose=0)

    s = next_state   
    
    # updates the exploration rate
    if epsilon > epsilon_min: epsilon *= epsilon_decay 
    
    print("Acao otima para cada canal:", target_f, "\n")
    
        
    """    
    print("Tabela do Q-value Maximo")
    print(Q_table, "\n")
    print("Acao otima para cada canal:", np.argmax(Q_table, axis=1), "\n")
    print("---------------------------", "\n")
    """
   

Time Slot    : 0
---------------------- 

Current State: [[1 0 0 0]]
Action       : 0
Next state   : [[0 1 0 0]]
Reward       : 1.0
Gameover     : False
---------------------- 

Acao otima para cada canal: [[1.1828909  0.27163616 0.1906301  0.28510207]] 

Time Slot    : 1
---------------------- 

Current State: [[0 1 0 0]]
Action       : 0
Next state   : [[0 0 1 0]]
Reward       : 1.0
Gameover     : False
---------------------- 

Acao otima para cada canal: [[1.2236385  0.26324275 0.27252445 0.2714973 ]] 

Time Slot    : 2
---------------------- 

Current State: [[0 0 1 0]]
Action       : 0
Next state   : [[0 0 0 1]]
Reward       : 1.0
Gameover     : False
---------------------- 

Acao otima para cada canal: [[1.2355078  0.2791794  0.15571105 0.3294887 ]] 

Time Slot    : 3
---------------------- 

Current State: [[0 0 0 1]]
Action       : 0
Next state   : [[1 0 0 0]]
Reward       : -1.0
Gameover     : False
---------------------- 

Acao otima para cada canal: [[-0.75903124  0.26572245

Acao otima para cada canal: [[-0.75638264  0.26096228  0.2087791   0.27959743]] 

Time Slot    : 40
---------------------- 

Current State: [[1 0 0 0]]
Action       : 3
Next state   : [[0 1 0 0]]
Reward       : 1.0
Gameover     : False
---------------------- 

Acao otima para cada canal: [[0.25640878 0.26551223 0.19092037 1.1856573 ]] 

Time Slot    : 41
---------------------- 

Current State: [[0 1 0 0]]
Action       : 2
Next state   : [[0 0 1 0]]
Reward       : -1.0
Gameover     : False
---------------------- 

Acao otima para cada canal: [[ 0.19548106  0.25896388 -0.77183753  0.27411443]] 

Time Slot    : 42
---------------------- 

Current State: [[0 0 1 0]]
Action       : 0
Next state   : [[0 0 0 1]]
Reward       : 1.0
Gameover     : False
---------------------- 

Acao otima para cada canal: [[1.23808    0.274642   0.15912025 0.32606387]] 

Time Slot    : 43
---------------------- 

Current State: [[0 0 0 1]]
Action       : 0
Next state   : [[1 0 0 0]]
Reward       : -1.0
Gameover

Acao otima para cada canal: [[1.190872   0.26241052 0.18757816 0.29105413]] 

Time Slot    : 73
---------------------- 

Current State: [[0 1 0 0]]
Action       : 1
Next state   : [[0 0 1 0]]
Reward       : 1.0
Gameover     : False
---------------------- 

Acao otima para cada canal: [[0.20107813 1.2311085  0.2637885  0.27757552]] 

Time Slot    : 74
---------------------- 

Current State: [[0 0 1 0]]
Action       : 0
Next state   : [[0 0 0 1]]
Reward       : 1.0
Gameover     : False
---------------------- 

Acao otima para cada canal: [[1.2408344  0.27159512 0.15946355 0.32558677]] 

Time Slot    : 75
---------------------- 

Current State: [[0 0 0 1]]
Action       : 3
Next state   : [[1 0 0 0]]
Reward       : 1.0
Gameover     : False
---------------------- 

Acao otima para cada canal: [[0.25360817 0.25759196 0.2085364  1.2463005 ]] 

Time Slot    : 76
---------------------- 

Current State: [[1 0 0 0]]
Action       : 2
Next state   : [[0 1 0 0]]
Reward       : 1.0
Gameover     : Fal